In [1]:
import pandas as pd
import os
import re
from glob import glob

In [2]:
# Parameters
# You can change this path if needed
years = [2018, 2019, 2021, 2022, 2023, 2024]

# Helper function to extract driver number from filename
def extract_driver_number(filename):
    match = re.search(r'driver_(\d+)', filename)
    return match.group(1) if match else None

# Process each year
for year in years:
    # Load lap data for the year
    lap_file = os.path.join(f"monaco_laps_{year}.csv")
    df_lap = pd.read_csv(lap_file)
    df_lap['Time'] = pd.to_timedelta(df_lap['Time'])

    # Prepare list to collect all car data for this year
    car_files = sorted(glob(os.path.join(f"car_data_{year}_driver_*.csv")))
    car_data_frames = []

    for car_file in car_files:
        driver_number = extract_driver_number(car_file)
        df_car = pd.read_csv(car_file)
        df_car['Time'] = pd.to_timedelta(df_car['Time'])
        df_car['DriverNumber'] = int(driver_number)
        car_data_frames.append(df_car)

    # Combine all car data for the year
    df_car = pd.concat(car_data_frames, ignore_index=True)

    # Annotate df_car with LapNumber
    df_car['LapNumber'] = None

    # Preprocess for efficiency
    df_lap = df_lap[['DriverNumber', 'LapNumber', 'Time']].copy()
    df_lap['Time'] = pd.to_timedelta(df_lap['Time'])
    df_lap = df_lap.sort_values(['DriverNumber', 'LapNumber'])

    # Iterate through each driver's laps to assign LapNumber in df_car
    for driver in df_lap['DriverNumber'].unique():
        driver_laps = df_lap[df_lap['DriverNumber'] == driver].sort_values('LapNumber')
        driver_car_data = df_car['DriverNumber'] == driver

        for i in range(len(driver_laps) - 1):
            lap_start = driver_laps.iloc[i]['Time']
            lap_end = driver_laps.iloc[i + 1]['Time']
            lap_number = driver_laps.iloc[i]['LapNumber']

            in_lap = driver_car_data & (df_car['Time'] >= lap_start) & (df_car['Time'] < lap_end)
            df_car.loc[in_lap, 'LapNumber'] = lap_number

        # Assign last lap for each driver
        last_lap = driver_laps.iloc[-1]
        in_last_lap = driver_car_data & (df_car['Time'] >= last_lap['Time'])
        df_car.loc[in_last_lap, 'LapNumber'] = last_lap['LapNumber']
    df_car = df_car.dropna(subset=['LapNumber'])

    # Save the annotated file for this year
    df_car.to_csv(os.path.join(f"car_data_with_lap_{year}.csv"), index=False)

In [3]:
years = [2018, 2019, 2021, 2022, 2023, 2024]
all_dfs = []

for year in years:
    print(f"Processing year {year}...")
    file_path = f"car_data_with_lap_{year}.csv"
    df = pd.read_csv(file_path)

    # Ensure correct dtypes
    df['Brake'] = df['Brake'].astype(bool)
    df['LapNumber'] = df['LapNumber'].astype(int)

    df = df[(df['nGear'] < 9)]

    # Aggregation
    agg_df = df.groupby(['DriverNumber', 'LapNumber']).agg({
        'RPM': 'mean',
        'nGear': 'mean',
        'Throttle': 'mean',
        'Brake': lambda x: x.sum(),  # True count
        'DRS': lambda x: x.isin([10, 12, 14]).sum()
    }).reset_index()

    #Saving as CSV
    output_file = f"aggregated_car_data_{year}.csv"
    agg_df.to_csv(output_file, index=False)

    try:
        df = pd.read_csv(output_file)
        df["Year"] = year  # optionally annotate year
        all_dfs.append(df)
    except FileNotFoundError:
        print(f"File for {year} not found. Skipping...")

df_all_years_car = pd.concat(all_dfs, ignore_index=True)

Processing year 2018...
Processing year 2019...
Processing year 2021...
Processing year 2022...
Processing year 2023...
Processing year 2024...


In [4]:
df_all_years_car

,DriverNumber,LapNumber,RPM,nGear,Throttle,Brake,DRS,Year
0,2,1,9507.728070,3.751462,45.207602,98,0,2018
1,2,2,9467.687500,3.797619,44.747024,107,15,2018
2,2,3,9421.166667,3.827381,43.125000,101,20,2018
3,2,4,9429.387387,3.843844,42.303303,102,18,2018
4,2,5,9452.302395,3.787425,41.700599,108,18,2018
...,...,...,...,...,...,...,...,...
8306,81,74,9448.278912,3.693878,48.397959,84,0,2024
8307,81,75,9520.927835,3.680412,49.233677,80,0,2024
8308,81,76,9501.748322,3.684564,48.345638,83,0,2024
8309,81,77,9522.473498,3.692580,48.823322,79,0,2024


In [5]:
df_lap_2018 = pd.read_csv('monaco_laps_2018.csv')
df_lap_2019 = pd.read_csv('monaco_laps_2019.csv')
df_lap_2021 = pd.read_csv('monaco_laps_2021.csv')
df_lap_2022 = pd.read_csv('monaco_laps_2022.csv')
df_lap_2023 = pd.read_csv('monaco_laps_2023.csv')
df_lap_2024 = pd.read_csv('monaco_laps_2024.csv')

In [6]:
df_laps = pd.concat([df_lap_2018, df_lap_2019, df_lap_2021, df_lap_2022, df_lap_2023, df_lap_2024])

In [7]:
df_laps

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year
0,0 days 00:08:19.100000,GAS,10,0 days 00:01:31.614000,1.0,NaN,NaN,NaN,NaN,0 days 00:00:39.994000,...,Toro Rosso,0 days 00:06:47.319000,NaN,2,10.0,False,NaN,False,False,2018
1,0 days 00:09:39.719000,GAS,10,0 days 00:01:20.619000,2.0,1.0,NaN,NaN,0 days 00:00:21.571000,0 days 00:00:37.371000,...,Toro Rosso,0 days 00:08:19.100000,NaN,21,10.0,False,NaN,False,True,2018
2,0 days 00:10:59.531000,GAS,10,0 days 00:01:19.812000,3.0,1.0,NaN,NaN,0 days 00:00:21.179000,0 days 00:00:37.273000,...,Toro Rosso,0 days 00:09:39.719000,NaN,1,10.0,False,NaN,False,True,2018
3,0 days 00:12:19.093000,GAS,10,0 days 00:01:19.562000,4.0,1.0,NaN,NaN,0 days 00:00:21.050000,0 days 00:00:37.245000,...,Toro Rosso,0 days 00:10:59.531000,NaN,1,10.0,False,NaN,False,True,2018
4,0 days 00:13:38.493000,GAS,10,0 days 00:01:19.400000,5.0,1.0,NaN,NaN,0 days 00:00:21.054000,0 days 00:00:37.039000,...,Toro Rosso,0 days 00:12:19.093000,NaN,1,10.0,False,NaN,False,True,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,0 days 03:14:09.170000,ZHO,24,0 days 00:01:20.585000,72.0,3.0,NaN,NaN,0 days 00:00:20.370000,0 days 00:00:36.805000,...,Kick Sauber,0 days 03:12:48.585000,NaN,1,16.0,False,NaN,False,True,2024
1222,0 days 03:15:24.403000,ZHO,24,0 days 00:01:15.233000,73.0,3.0,NaN,NaN,0 days 00:00:19.940000,0 days 00:00:35.135000,...,Kick Sauber,0 days 03:14:09.170000,NaN,1,16.0,False,NaN,False,True,2024
1223,0 days 03:16:44.876000,ZHO,24,0 days 00:01:20.473000,74.0,3.0,NaN,NaN,0 days 00:00:21.051000,0 days 00:00:37.048000,...,Kick Sauber,0 days 03:15:24.403000,NaN,1,16.0,False,NaN,False,True,2024
1224,0 days 03:17:59.594000,ZHO,24,0 days 00:01:14.718000,75.0,3.0,NaN,NaN,0 days 00:00:19.751000,0 days 00:00:34.990000,...,Kick Sauber,0 days 03:16:44.876000,NaN,1,16.0,False,NaN,False,True,2024


In [8]:
merged_df = pd.merge(
    df_all_years_car,
    df_laps,
    on=['DriverNumber', 'LapNumber', 'Year'],
    how='right'
)

In [9]:
merged_df

,DriverNumber,LapNumber,RPM,nGear,Throttle,Brake,DRS,Year,Time,Driver,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,10,1,9531.494048,3.413690,46.101190,99,0,2018,0 days 00:08:19.100000,GAS,...,True,Toro Rosso,0 days 00:06:47.319000,NaN,2,10.0,False,NaN,False,False
1,10,2,9394.374251,3.592814,47.080838,93,16,2018,0 days 00:09:39.719000,GAS,...,False,Toro Rosso,0 days 00:08:19.100000,NaN,21,10.0,False,NaN,False,True
2,10,3,9429.505952,3.583333,48.086310,89,24,2018,0 days 00:10:59.531000,GAS,...,False,Toro Rosso,0 days 00:09:39.719000,NaN,1,10.0,False,NaN,False,True
3,10,4,9437.838906,3.589666,47.723404,85,22,2018,0 days 00:12:19.093000,GAS,...,False,Toro Rosso,0 days 00:10:59.531000,NaN,1,10.0,False,NaN,False,True
4,10,5,9427.572727,3.600000,46.830303,92,19,2018,0 days 00:13:38.493000,GAS,...,False,Toro Rosso,0 days 00:12:19.093000,NaN,1,10.0,False,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8306,24,72,9382.875887,3.932624,50.872340,90,0,2024,0 days 03:14:09.170000,ZHO,...,True,Kick Sauber,0 days 03:12:48.585000,NaN,1,16.0,False,NaN,False,True
8307,24,73,8971.025559,3.750799,43.853035,102,0,2024,0 days 03:15:24.403000,ZHO,...,True,Kick Sauber,0 days 03:14:09.170000,NaN,1,16.0,False,NaN,False,True
8308,24,74,9461.837545,4.014440,52.931408,85,0,2024,0 days 03:16:44.876000,ZHO,...,True,Kick Sauber,0 days 03:15:24.403000,NaN,1,16.0,False,NaN,False,True
8309,24,75,9103.670175,3.870175,47.568421,88,0,2024,0 days 03:17:59.594000,ZHO,...,True,Kick Sauber,0 days 03:16:44.876000,NaN,1,16.0,False,NaN,False,True


In [10]:
merged_df.columns

Index(['DriverNumber', 'LapNumber', 'RPM', 'nGear', 'Throttle', 'Brake', 'DRS',
       'Year', 'Time', 'Driver', 'LapTime', 'Stint', 'PitOutTime', 'PitInTime',
       'Sector1Time', 'Sector2Time', 'Sector3Time', 'Sector1SessionTime',
       'Sector2SessionTime', 'Sector3SessionTime', 'SpeedI1', 'SpeedI2',
       'SpeedFL', 'SpeedST', 'IsPersonalBest', 'Compound', 'TyreLife',
       'FreshTyre', 'Team', 'LapStartTime', 'LapStartDate', 'TrackStatus',
       'Position', 'Deleted', 'DeletedReason', 'FastF1Generated',
       'IsAccurate'],
      dtype='object')

In [11]:
merged_df.to_csv('cars_laps_monaco_all_years.csv', index=False)

In [12]:
merged_df = pd.read_csv('cars_laps_monaco_all_years.csv')

In [13]:
merged_df[merged_df['Year']==2018]

,DriverNumber,LapNumber,RPM,nGear,Throttle,Brake,DRS,Year,Time,Driver,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,10,1,9531.494048,3.413690,46.101190,99,0,2018,0 days 00:08:19.100000,GAS,...,True,Toro Rosso,0 days 00:06:47.319000,NaN,2,10.0,False,NaN,False,False
1,10,2,9394.374251,3.592814,47.080838,93,16,2018,0 days 00:09:39.719000,GAS,...,False,Toro Rosso,0 days 00:08:19.100000,NaN,21,10.0,False,NaN,False,True
2,10,3,9429.505952,3.583333,48.086310,89,24,2018,0 days 00:10:59.531000,GAS,...,False,Toro Rosso,0 days 00:09:39.719000,NaN,1,10.0,False,NaN,False,True
3,10,4,9437.838906,3.589666,47.723404,85,22,2018,0 days 00:12:19.093000,GAS,...,False,Toro Rosso,0 days 00:10:59.531000,NaN,1,10.0,False,NaN,False,True
4,10,5,9427.572727,3.600000,46.830303,92,19,2018,0 days 00:13:38.493000,GAS,...,False,Toro Rosso,0 days 00:12:19.093000,NaN,1,10.0,False,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,9,74,9462.390909,3.724242,48.800000,86,0,2018,0 days 01:45:38.757000,ERI,...,True,Sauber,0 days 01:44:19.686000,NaN,1,11.0,False,NaN,False,True
1509,9,75,9509.975535,3.752294,49.474006,87,0,2018,0 days 01:46:57.161000,ERI,...,True,Sauber,0 days 01:45:38.757000,NaN,1,11.0,False,NaN,False,True
1510,9,76,9446.483180,3.755352,48.633028,81,0,2018,0 days 01:48:15.009000,ERI,...,True,Sauber,0 days 01:46:57.161000,NaN,1,11.0,False,NaN,False,True
1511,9,77,9328.184848,3.703030,45.909091,89,0,2018,0 days 01:49:32.973000,ERI,...,True,Sauber,0 days 01:48:15.009000,NaN,1,11.0,False,NaN,False,True


In [14]:
import pandas as pd

years = [2018, 2019, 2021, 2022, 2023, 2024]

for year in years:
    try:
        # Load data
        lap_file = f"monaco_laps_{year}.csv"
        weather_file = f"weather_{year}.csv"

        df_lap = pd.read_csv(lap_file)
        df_weather = pd.read_csv(weather_file)

        df_lap["Time"] = pd.to_timedelta(df_lap["Time"])
        df_weather["Time"] = pd.to_timedelta(df_weather["Time"])
        df_weather["LapNumber"] = None

        # Step 1: Create lap boundaries from min Time per LapNumber
        lap_boundaries = df_lap.groupby("LapNumber")["Time"].min().reset_index().sort_values("LapNumber")

        # Step 2: Iterate over lap boundaries to annotate weather data
        for i in range(len(lap_boundaries) - 1):
            lap_num = lap_boundaries.iloc[i]["LapNumber"]
            lap_start = lap_boundaries.iloc[i]["Time"]
            lap_end = lap_boundaries.iloc[i + 1]["Time"]

            mask = (df_weather["Time"] >= lap_start) & (df_weather["Time"] < lap_end)
            df_weather.loc[mask, "LapNumber"] = lap_num

        # Final lap — anything after last lap start
        last_lap = lap_boundaries.iloc[-1]
        mask = df_weather["Time"] >= last_lap["Time"]
        df_weather.loc[mask, "LapNumber"] = last_lap["LapNumber"]

        # Drop rows without LapNumber
        df_weather = df_weather.dropna(subset=["LapNumber"])

        # Save output
        output_file = f"weather_with_lap_{year}.csv"
        df_weather.to_csv(output_file, index=False)
        print(f"✅ Annotated weather for {year} → {output_file}")

    except Exception as e:
        print(f"❌ Error processing {year}: {e}")

✅ Annotated weather for 2018 → weather_with_lap_2018.csv
✅ Annotated weather for 2019 → weather_with_lap_2019.csv
✅ Annotated weather for 2021 → weather_with_lap_2021.csv
✅ Annotated weather for 2022 → weather_with_lap_2022.csv
✅ Annotated weather for 2023 → weather_with_lap_2023.csv
✅ Annotated weather for 2024 → weather_with_lap_2024.csv


In [15]:
# Path and years
#base_dir = r"D:\MSIM 4th sem\PR\2025Spring_Final_F1\Fastf1"
years = [2018, 2019, 2021, 2022, 2023, 2024]

# Assume merged_df is already in memory
# Ensure Time columns are in timedelta format
merged_df["Time"] = pd.to_timedelta(merged_df["Time"])

for year in years:
    try:
        #weather_file = os.path.join(base_dir, f"weather_{year}.csv")
        weather_file = f"weather_{year}.csv"
        df_weather = pd.read_csv(weather_file)
        df_weather["Time"] = pd.to_timedelta(df_weather["Time"])
        df_weather["LapNumber"] = None

        year_laps = merged_df[merged_df["Year"] == year]

        for _, lap in year_laps.iterrows():
            mask = (df_weather["Time"] >= lap["Time"]) & (df_weather["Time"] < lap["Time"])
            df_weather.loc[mask, "LapNumber"] = lap["LapNumber"]

        df_weather = df_weather.dropna(subset=["LapNumber"])
        #output_file = os.path.join(base_dir, f"weather_with_lap_{year}.csv")
        output_file = f"weather_with_lap{year}.csv"
        df_weather.to_csv(output_file, index=False)
        print(f"✅ Annotated weather data saved: {output_file}")

    except Exception as e:
        print(f"❌ Error processing weather data for {year}: {e}")


✅ Annotated weather data saved: weather_with_lap2018.csv
✅ Annotated weather data saved: weather_with_lap2019.csv
✅ Annotated weather data saved: weather_with_lap2021.csv
✅ Annotated weather data saved: weather_with_lap2022.csv
✅ Annotated weather data saved: weather_with_lap2023.csv
✅ Annotated weather data saved: weather_with_lap2024.csv


In [16]:
pd.read_csv('weather_with_lap_2018.csv')['Rainfall'].sum()

14

In [17]:

years = [2018, 2019, 2021, 2022, 2023, 2024]

aggregated_weather_dfs = []

for year in years:
    file_path = os.path.join(f"weather_with_lap_{year}.csv")
    try:
        df = pd.read_csv(file_path)

        # Drop Time column if it exists
        if "Time" in df.columns:
            df = df.drop(columns=["Time"])

        # Define aggregation rules
        agg_funcs = {col: 'mean' for col in df.columns if col not in ['LapNumber', 'Rainfall']}
        if 'Rainfall' in df.columns:
            agg_funcs['Rainfall'] = 'sum'

        # Group by LapNumber and aggregate
        df_agg = df.groupby("LapNumber").agg(agg_funcs).reset_index()
        df_agg["Year"] = year
        aggregated_weather_dfs.append(df_agg)

    except Exception as e:
        print(f"❌ Could not process year {year}: {e}")

# Concatenate all years
df_weather_all_years = pd.concat(aggregated_weather_dfs, ignore_index=True)

# Optional: Save to CSV
df_weather_all_years.to_csv(os.path.join("aggregated_weather_all_years.csv"), index=False)
print("✅ Aggregation complete and saved!")


✅ Aggregation complete and saved!


In [18]:
df_weather_all_years

,LapNumber,AirTemp,Humidity,Pressure,TrackTemp,WindDirection,WindSpeed,Rainfall,Year
0,1.0,25.70,51.0,1015.65,33.75,193.0,0.70,1,2018
1,2.0,26.00,50.7,1015.60,33.90,181.0,0.60,1,2018
2,3.0,26.00,50.7,1015.60,33.90,181.0,0.60,1,2018
3,4.0,26.10,50.0,1015.60,33.90,283.0,1.00,1,2018
4,5.0,26.60,48.6,1015.60,33.60,232.0,1.65,2,2018
...,...,...,...,...,...,...,...,...,...
448,74.0,21.70,61.0,1018.40,40.80,214.0,0.70,0,2024
449,75.0,21.70,61.0,1018.40,40.60,353.0,0.60,0,2024
450,76.0,21.75,61.0,1018.40,40.75,221.5,1.05,0,2024
451,77.0,21.80,61.0,1018.40,40.90,284.0,0.80,0,2024


In [19]:
merged_df

,DriverNumber,LapNumber,RPM,nGear,Throttle,Brake,DRS,Year,Time,Driver,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,10,1,9531.494048,3.413690,46.101190,99,0,2018,0 days 00:08:19.100000,GAS,...,True,Toro Rosso,0 days 00:06:47.319000,NaN,2,10.0,False,NaN,False,False
1,10,2,9394.374251,3.592814,47.080838,93,16,2018,0 days 00:09:39.719000,GAS,...,False,Toro Rosso,0 days 00:08:19.100000,NaN,21,10.0,False,NaN,False,True
2,10,3,9429.505952,3.583333,48.086310,89,24,2018,0 days 00:10:59.531000,GAS,...,False,Toro Rosso,0 days 00:09:39.719000,NaN,1,10.0,False,NaN,False,True
3,10,4,9437.838906,3.589666,47.723404,85,22,2018,0 days 00:12:19.093000,GAS,...,False,Toro Rosso,0 days 00:10:59.531000,NaN,1,10.0,False,NaN,False,True
4,10,5,9427.572727,3.600000,46.830303,92,19,2018,0 days 00:13:38.493000,GAS,...,False,Toro Rosso,0 days 00:12:19.093000,NaN,1,10.0,False,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8306,24,72,9382.875887,3.932624,50.872340,90,0,2024,0 days 03:14:09.170000,ZHO,...,True,Kick Sauber,0 days 03:12:48.585000,NaN,1,16.0,False,NaN,False,True
8307,24,73,8971.025559,3.750799,43.853035,102,0,2024,0 days 03:15:24.403000,ZHO,...,True,Kick Sauber,0 days 03:14:09.170000,NaN,1,16.0,False,NaN,False,True
8308,24,74,9461.837545,4.014440,52.931408,85,0,2024,0 days 03:16:44.876000,ZHO,...,True,Kick Sauber,0 days 03:15:24.403000,NaN,1,16.0,False,NaN,False,True
8309,24,75,9103.670175,3.870175,47.568421,88,0,2024,0 days 03:17:59.594000,ZHO,...,True,Kick Sauber,0 days 03:16:44.876000,NaN,1,16.0,False,NaN,False,True


In [20]:
merged_df_1 = pd.merge(
    df_weather_all_years,
    merged_df,
    on=['LapNumber', 'Year'],
    how='right'
)

In [21]:
merged_df_1

,LapNumber,AirTemp,Humidity,Pressure,TrackTemp,WindDirection,WindSpeed,Rainfall,Year,DriverNumber,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,1.0,25.70,51.0,1015.65,33.75,193.0,0.70,1.0,2018,10,...,True,Toro Rosso,0 days 00:06:47.319000,NaN,2,10.0,False,NaN,False,False
1,2.0,26.00,50.7,1015.60,33.90,181.0,0.60,1.0,2018,10,...,False,Toro Rosso,0 days 00:08:19.100000,NaN,21,10.0,False,NaN,False,True
2,3.0,26.00,50.7,1015.60,33.90,181.0,0.60,1.0,2018,10,...,False,Toro Rosso,0 days 00:09:39.719000,NaN,1,10.0,False,NaN,False,True
3,4.0,26.10,50.0,1015.60,33.90,283.0,1.00,1.0,2018,10,...,False,Toro Rosso,0 days 00:10:59.531000,NaN,1,10.0,False,NaN,False,True
4,5.0,26.60,48.6,1015.60,33.60,232.0,1.65,2.0,2018,10,...,False,Toro Rosso,0 days 00:12:19.093000,NaN,1,10.0,False,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8306,72.0,21.60,61.5,1018.40,40.50,211.5,0.70,0.0,2024,24,...,True,Kick Sauber,0 days 03:12:48.585000,NaN,1,16.0,False,NaN,False,True
8307,73.0,21.70,61.0,1018.40,40.80,293.0,1.10,0.0,2024,24,...,True,Kick Sauber,0 days 03:14:09.170000,NaN,1,16.0,False,NaN,False,True
8308,74.0,21.70,61.0,1018.40,40.80,214.0,0.70,0.0,2024,24,...,True,Kick Sauber,0 days 03:15:24.403000,NaN,1,16.0,False,NaN,False,True
8309,75.0,21.70,61.0,1018.40,40.60,353.0,0.60,0.0,2024,24,...,True,Kick Sauber,0 days 03:16:44.876000,NaN,1,16.0,False,NaN,False,True


In [22]:
len(merged_df)

8311

In [23]:
merged_df_1.to_csv('regression_w_l_c.csv', index=False)

In [24]:
merged_df_1.columns

Index(['LapNumber', 'AirTemp', 'Humidity', 'Pressure', 'TrackTemp',
       'WindDirection', 'WindSpeed', 'Rainfall', 'Year', 'DriverNumber', 'RPM',
       'nGear', 'Throttle', 'Brake', 'DRS', 'Time', 'Driver', 'LapTime',
       'Stint', 'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time',
       'Sector3Time', 'Sector1SessionTime', 'Sector2SessionTime',
       'Sector3SessionTime', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST',
       'IsPersonalBest', 'Compound', 'TyreLife', 'FreshTyre', 'Team',
       'LapStartTime', 'LapStartDate', 'TrackStatus', 'Position', 'Deleted',
       'DeletedReason', 'FastF1Generated', 'IsAccurate'],
      dtype='object')